In [ ]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

import os.path

We need to read the Recon3D model, which will be used as a reference.

In [ ]:
model = cobra.io.read_sbml_model(os.path.join('models','Recon3D.xml'))

We will prepare two dictionaries to connect gene ids with gene names and vice versa:

In [ ]:
gene_id_name = {} # from gene id to gene name
gene_name_id = {} # from gene name to gene id

for g in model.genes[1:]:
    gene_id_name[g.id] = g.name
    gene_name_id[g.name] = g.id

In [ ]:
list(gene_id_name.items())[:5] # the first 5 elements of the dictionary connecting gene ids with gene names

In [ ]:
list(gene_name_id.items())[:5] # the first 5 elements of the dictionary connecting gene names with gene ids

Next, we will read the data, that we prepared in the previous step:

In [ ]:
df_control = pd.read_csv(os.path.join('data','data_control.txt'))
df_kd = pd.read_csv(os.path.join('data','data_kd.txt'))

We will assign gene ids to the genes:

In [ ]:
df_control['gene_id'] = df_control['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")
df_kd['gene_id'] = df_kd['gene'].map(lambda x: gene_name_id[x] if x in gene_name_id else "")

In [ ]:
df_control

and we will keep only the genes that are present in the model:

In [ ]:
df_control = df_control[df_control['gene_id'] != ""]
df_kd = df_kd[df_kd['gene_id'] != ""]

In [ ]:
len(df_control), len(df_kd)

In [ ]:
df_control.describe()

In [ ]:
df_kd.describe()

### Integration of transcriptomics data into the model

We will employ a method called CORDA (Cost Optimization Reaction Dependency Assessment):

* [https://resendislab.github.io/corda/](https://resendislab.github.io/corda/)
* [https://github.com/resendislab/corda](https://github.com/resendislab/corda)

We will use a function that requires the specification of `confidence` scores for each reaction in a model:
```
Allowed confidence values are -1 (absent/do not include), 0 (unknown), 1 (low confidence), 2 (medium confidence) and 3 (high confidence).
```

`confidence` can be prepared using the CORDA function `reaction_confidence`. This function will project gene expression levels (on the same scale as above) into reaction activity levels using the GPR rules in a model.


Additionally we can require for the model to keep biomass production reaction active:
```
met_prod (Optional[list]): Additional metabolic targets that have to be
            achieved by the model. Can be a single object or list of objects.
            List elements can be given in various forms:
            (1) A string naming a metabolite in the model will ensure that the
            metabolite can be produced.
            For instance: met_prod = ["pi_c", "atp_c"]
            (2) A dictionary of metabolite -> int will define an irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = {"adp_c": -1, "pi_c": -1, "atp_c": 1}
            (3) A string representation of a reversible or irreversible
            reaction that must be able to carry flux.
            For instance: met_prod = "adp_c + pi_c -> atp_c"

```
Example of preparation of `met_prod` string:
```Python
met_prod = model.reactions.BIOMASS_maintenance.build_reaction_string()
met_prod = met_prod.replace("-->", "->")
```


In [ ]:
biomass = model.reactions.BIOMASS_maintenance.build_reaction_string().replace("-->", "->")
biomass

### Projection of gene expression levels to reaction activities

In [ ]:
from corda import reaction_confidence
from corda import CORDA

We will prepare a function that will project gene expression levels to values -1, 0, 1, 2 and 3. The projection will be based on predefined thresholds.

In [ ]:
def val_to_score(val, thr0, thr1, thr2, thr3):
    if val < thr0:
        return -1
    elif val < thr1:
        return 0
    elif val < thr2:
        return 1
    elif val < thr3:
        return 2
    else:
        return 3

#### Reconstruction parameters
The thresholds will be defined on the basis of gene expression levels in each sample. This can be achieved with different strategies ([further reading](https://doi.org/10.1371/journal.pcbi.1007185)). In our case, we will observe percentile scores.

In [ ]:
require_biomass = True
perc0 = 25 # below this ... inactive
perc1 = 50 # else, below this ... not sure
perc2 = 75 # else, below this ...  low confidence
perc3 = 95 # else, below this ... medium confidence
# else high confidence (above perc3)

In [ ]:
folder = "models"
folder

### Control
The thresholds will be defined for each group separately:

In [ ]:
thr0 = np.percentile(df_control.value, perc0)
thr1 = np.percentile(df_control.value, perc1)
thr2 = np.percentile(df_control.value, perc2)
thr3 = np.percentile(df_control.value, perc3)

First we need to project gene expression levels to predefined groups:

In [ ]:
#gene_conf_control = {g_id:-1 if val < thr0 else 1 if val < thr1 else 2 if val < thr2 else 3 for (g_id, val) in zip(df_control['gene_id'], df_control['value'])}
gene_conf_control = {g_id : val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df_control['gene_id'], df_control['value'])}

Next we can employ the CORDA function `reaction_confidence`. This function will use to gene groups to classify reactions in a model into the same groups (with the application of GPR rules).

In [ ]:
conf_control = {}
for r in model.reactions:
    conf_control[r.id] = reaction_confidence(r, gene_conf_control)

Finally, we can execute the data integration step.

**CAUTION: this step is time consuming! You can skip the cells below in read the data from a file that was prepared in advance.**

In [ ]:
if require_biomass:
    opt_control = CORDA(model, conf_control, met_prod = biomass)
else:
    opt_control = CORDA(model, conf_control)
opt_control.build()
print(opt_control)

First we need to translate the model into a COBRA model

In [ ]:
model_control = opt_control.cobra_model()

If biomass function should also be active in the final model

In [ ]:
if require_biomass:
    model_control.add_reactions([model.reactions.BIOMASS_maintenance])
    model_control.objective=model.objective

Finally, we store the model into an SBML file:

In [ ]:
cobra.io.write_sbml_model(model_control, os.path.join(f'{folder}','model_control.xml'))

**Reading the results from a file (in a case the steps above were skipped).**

In [ ]:
model_control = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_control.xml'))

In [ ]:
model_control.summary()

### Knockdown

We can repeat the same procedure on the dataset with silenced oncogene:

In [ ]:
thr0 = np.percentile(df_kd.value, perc0)
thr1 = np.percentile(df_kd.value, perc1)
thr2 = np.percentile(df_kd.value, perc2)
thr3 = np.percentile(df_kd.value, perc3)

In [ ]:
gene_conf_kd = {g_id : val_to_score(val, thr0, thr1, thr2, thr3) for (g_id, val) in zip(df_kd['gene_id'], df_kd['value'])}

In [ ]:
conf_kd = {}
for r in model.reactions:
    conf_kd[r.id] = reaction_confidence(r, gene_conf_kd)

**CAUTION: this step is time consuming! You can skip the cells below in read the data from a file that was prepared in advance.**

In [ ]:
if require_biomass:
    opt_kd = CORDA(model, conf_kd, met_prod = biomass)
else:
    opt_kd = CORDA(model, conf_kd)
opt_kd.build()
print(opt_kd)

In [ ]:
model_kd = opt_kd.cobra_model()

In [ ]:
if require_biomass:
    model_kd.add_reactions([model.reactions.BIOMASS_maintenance])
    model_kd.objective=model.objective   

In [ ]:
cobra.io.write_sbml_model(model_kd, os.path.join(f'{folder}','model_kd.xml'))

**Reading the results from a file (in a case the steps above were skipped).**

In [ ]:
model_kd = cobra.io.read_sbml_model(os.path.join(f'{folder}','model_kd.xml'))

In [ ]:
model_kd.summary()